In [42]:
import os
import sqlite3
import csv
import json
import pandas as pd
import numpy as np
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Float, DateTime, String


def contact_info_address(val):
    new_dict = json.loads(val)
    mailing_address = new_dict.get('mailing_address')
    return mailing_address


def contact_info_email(val):
    new_dict = json.loads(val)
    email = new_dict.get('email')
    return email


def convert_to_int(val):
    new_val = int(val.split('.')[0])
    return new_val


In [43]:
conn = sqlite3.connect("subscriber-pipeline-starter-kit/dev/cademycode.db")

df_students = pd.read_sql_query("SELECT DISTINCT * FROM cademycode_students", conn)
df_courses = pd.read_sql_query("SELECT DISTINCT * FROM cademycode_courses", conn)
df_student_jobs = pd.read_sql_query("SELECT DISTINCT * FROM cademycode_student_jobs", conn)

df_students['dob'] = df_students.dob.astype('datetime64[ns]')

df_students['mailing_address'] = df_students.contact_info.apply(contact_info_address)
df_students['email'] = df_students.contact_info.apply(contact_info_email)
df_students = df_students.drop(columns=['contact_info'])

df_students['job_id'] = df_students.job_id.fillna('0.0')
df_students['job_id'] = df_students.job_id.apply(convert_to_int)

df_students['current_career_path_id'] = df_students.current_career_path_id.fillna('11.0')
df_students['current_career_path_id'] = df_students.current_career_path_id.apply(convert_to_int)
df_students = df_students.rename(columns={'current_career_path_id': 'career_path_id'})

df_students['num_course_taken'] = df_students.num_course_taken.fillna('0.0')
df_students['num_course_taken'] = df_students.num_course_taken.apply(convert_to_int)

df_students['time_spent_hrs'] = df_students.time_spent_hrs.fillna('0.0')
df_students['time_spent_hrs'] = df_students.time_spent_hrs.astype(np.float64)

df_courses.loc[len(df_courses)] = [11, 'Other', 20]

df_final = pd.merge(df_students, df_student_jobs, how='inner', on='job_id')
df_final = pd.merge(df_final, df_courses, how='inner', on='career_path_id')

df_final = df_final.reindex(columns=['uuid', 'name', 'dob', 'sex', 'mailing_address', 
                   'email', 'job_id', 'job_category', 'avg_salary', 
                   'num_course_taken', 'time_spent_hrs', 'career_path_id', 
                   'career_path_name', 'hours_to_complete'])

final_cols = {'uuid': 'user_id', 'name': 'full_name', 'dob': 'date_of_birth'}
df_final = df_final.rename(columns=final_cols)


In [44]:
df_final.head()


,user_id,full_name,date_of_birth,sex,mailing_address,email,job_id,job_category,avg_salary,num_course_taken,time_spent_hrs,career_path_id,career_path_name,hours_to_complete
0,1,Annabelle Avery,1943-07-03,F,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com,7,HR,80000,6,4.99,1,data scientist,20
1,15,Tamala Sears,1942-06-01,F,"868 Hazy Crossing, Bethania, Washington, 08205",tamala4408@woohoo.com,7,HR,80000,13,4.64,1,data scientist,20
2,24,Eartha Weeks,1954-12-05,F,"192 Cedar Vale, Hersey village, Illinois, 68184",eartha_weeks7248@hmail.com,7,HR,80000,2,9.86,1,data scientist,20
3,222,Ali de Kock,1960-03-12,M,"P.O. Box 63561, Arnett, Mississippi, 26124",dekock3427@coldmail.com,7,HR,80000,4,14.87,1,data scientist,20
4,306,Joesph Vermijn,1962-09-24,M,"92 W End Manor, Livermore, Missouri, 46461",vermijn4861@coldmail.com,7,HR,80000,8,9.66,1,data scientist,20


In [45]:
df_final.describe()


,user_id,date_of_birth,job_id,avg_salary,num_course_taken,time_spent_hrs,career_path_id,hours_to_complete
count,5000.000000,5000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2500.500000,1973-03-15 17:21:59.040000,4.16800,87189.400000,7.155400,10.435382,5.999000,21.596600
min,1.000000,1942-01-10 00:00:00,0.00000,10000.000000,0.000000,0.000000,1.000000,12.000000
25%,1250.750000,1957-03-07 18:00:00,2.00000,66000.000000,3.000000,3.680000,3.000000,18.000000
50%,2500.500000,1972-12-07 12:00:00,4.00000,86000.000000,7.000000,9.665000,6.000000,20.000000
75%,3750.250000,1988-10-30 00:00:00,6.00000,110000.000000,11.000000,15.840000,9.000000,27.000000
max,5000.000000,2004-12-28 00:00:00,8.00000,135000.000000,15.000000,35.980000,11.000000,35.000000
std,1443.520003,NaN,2.15107,29295.418959,4.784415,7.946934,3.175502,6.069599


In [46]:
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            5000 non-null   int64         
 1   full_name          5000 non-null   object        
 2   date_of_birth      5000 non-null   datetime64[ns]
 3   sex                5000 non-null   object        
 4   mailing_address    5000 non-null   object        
 5   email              5000 non-null   object        
 6   job_id             5000 non-null   int64         
 7   job_category       5000 non-null   object        
 8   avg_salary         5000 non-null   int64         
 9   num_course_taken   5000 non-null   int64         
 10  time_spent_hrs     5000 non-null   float64       
 11  career_path_id     5000 non-null   int64         
 12  career_path_name   5000 non-null   object        
 13  hours_to_complete  5000 non-null   int64         
dtypes: datet

In [47]:
df_final.to_csv("data_dev/subscriber_data_clean.csv", sep="|", index=False)


In [48]:
Base = declarative_base()


class Subscriber(Base):

    __tablename__ = "subscribers"

    user_id = Column(Integer, primary_key=True)
    full_name = Column(String)
    date_of_birth = Column(DateTime)
    sex = Column(String)
    mailing_address = Column(String)
    email = Column(String)
    job_id = Column(Integer)
    job_category = Column(String)
    avg_salary = Column(Integer)
    num_course_taken = Column(Integer)
    time_spent_hrs = Column(Float)
    career_path_id = Column(Integer)
    career_path_name = Column(String)
    hours_to_complete = Column(Integer)
    

os.remove("data_dev/cademycode_analytics.db")
engine = create_engine("sqlite:///data_dev/cademycode_analytics.db")
Base.metadata.create_all(engine)

conn_new = sqlite3.connect("data_dev/cademycode_analytics.db")
df_final.to_sql(name='subscribers', con=conn_new, if_exists='append', index=False)


5000